In [155]:
from PIL import Image
import numpy as np

# 1. Приведение изображения к raw формату

In [156]:
def imgtoraw(img):
    imgar = np.array(img)
    return imgar

# 2. Конвертация RGB в YCbCr

In [3]:
def rgbtoycbcr(arr):
    y = np.zeros(len(arr)*len(arr[0]))
    Cb = np.zeros(len(arr)*len(arr[0]))
    Cr = np.zeros(len(arr)*len(arr[0]))
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            y[j+i*len(arr[0])] = 0.299 * arr[i][j][0] + 0.587 * arr[i][j][1] + 0.114 * arr[i][j][2] + 16
            Cb[j+i*len(arr[0])] = -0.148 * arr[i][j][0] - 0.29 * arr[i][j][1] + 0.439 * arr[i][j][2] + 128
            Cr[j+i*len(arr[0])] = 0.439 * arr[i][j][0] - 0.367 * arr[i][j][1] + 0.07 * arr[i][j][2] + 128
    return y,Cb,Cr

img = Image.open("lena.png")
img = img.resize((400,400))
arr = imgtoraw(img)
y,Cb,Cr = rgbtoycbcr(arr)
y1 = np.reshape(y, (len(arr), len(arr[0])))
Cb1 = np.reshape(Cb, (len(arr), len(arr[0])))
Cr1 = np.reshape(Cr, (len(arr), len(arr[0])))
imy = Image.fromarray(y1.astype(np.uint8))
imcb = Image.fromarray(Cb1.astype(np.uint8))
imcr = Image.fromarray(Cr1.astype(np.uint8))
res_ycbcr = []
for i in range(len(y)):
    res_ycbcr.append([y[i],Cb[i],Cr[i]])
res_ycbcr = np.array(res_ycbcr)
res_ycbcr = np.reshape(res_ycbcr,(len(arr),len(arr[0]),3))
immg = Image.fromarray(res_ycbcr.astype(np.uint8), mode='YCbCr')

img.show()
imy.show()
imcb.show()
imcr.show()
immg.show()

img.close()
imy.close()
imcb.close()
imcr.close()
immg.close()

# 3. Конвертация YCbCr в RGB

In [ ]:
def ycbcrtorgb(arr):
    r = np.zeros(len(arr)*len(arr[0]))
    g = np.zeros(len(arr)*len(arr[0]))
    b = np.zeros(len(arr)*len(arr[0]))
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            r[j+i*len(arr[0])] = (arr[i][j][0]-16) * 0.774 - 0.456 * (arr[i][j][1]-128) + 1.597 * (arr[i][j][2]-128)
            g[j+i*len(arr[0])] = (arr[i][j][0]-16) * 1.116 - 0.16 * (arr[i][j][1]-128) - 0.814 * (arr[i][j][2]-128)
            b[j+i*len(arr[0])] = (arr[i][j][0]-16) * 0.998 + 2.019 * (arr[i][j][1]-128) + 9.12 * (arr[i][j][2]-128)
    return r,g,b

R,G,B = ycbcrtorgb(res_ycbcr)
r = np.reshape(R, (len(res_ycbcr), len(res_ycbcr[0])))
g = np.reshape(G, (len(res_ycbcr), len(res_ycbcr[0])))
b = np.reshape(B, (len(res_ycbcr), len(res_ycbcr[0])))
imr = Image.fromarray(r.astype(np.uint8))
res_rgb = []
for j in range(len(R)):
    res_rgb.append([R[j],G[j],B[j]])
res_rgb = np.array(res_rgb)
res_rgb = np.reshape(res_rgb,(len(res_ycbcr),len(res_ycbcr[0]), 3))
immg = Image.fromarray(res_rgb.astype(np.uint8), mode='RGB')

imr.show()
immg.show()

imr.close()
immg.close()

# 4. Обход матрицы зигзагом

In [141]:
def zigzag_order(matrix):

    rows, cols = matrix.shape
    result = np.zeros(rows * cols, dtype=int)
    index = -1
    bound = rows + cols - 1
    for i in range(bound):
        if i % 2 == 0:
            r = i if i < rows else rows - 1
            c = 0 if i < rows else i - rows + 1
            while r >= 0 and c < cols:
                index += 1
                result[index] = matrix[r, c]
                r -= 1
                c += 1
        else:
            r = 0 if i < cols else i - cols + 1
            c = i if i < cols else cols - 1
            while r < rows and c >= 0:
                index += 1
                result[index] = matrix[r, c]
                r += 1
                c -= 1
    return result

matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
]

print(zigzag_order(np.array(matrix)))


[ 1  2  5  9  6  3  4  7 10 11  8 12]


# 5. Обратная сборка матрицы после обхода зигзагом

In [144]:
def inverse_zigzag_order(array, rows, cols):
    matrix = np.zeros((rows, cols), dtype=int)
    index = -1
    bound = rows + cols - 1
    for i in range(bound):
        if i % 2 == 0:
            r = i if i < rows else rows - 1
            c = 0 if i < rows else i - rows + 1
            while r >= 0 and c < cols:
                index += 1
                matrix[r, c] = array[index]
                r -= 1
                c += 1
        else:
            r = 0 if i < cols else i - cols + 1
            c = i if i < cols else cols - 1
            while r < rows and c >= 0:
                index += 1
                matrix[r, c] = array[index]
                r += 1
                c -= 1
    return matrix

print(inverse_zigzag_order(zigzag_order(np.array(matrix)), 3, 4))

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


# 6. Даунсэмплинг

In [5]:
def downsample_image(image, Cx, Cy, method='remove'):
    N, M = image.shape
    new_N, new_M = N // Cx, M // Cy
    
    if method == 'remove':
        return image[::Cx, ::Cy]

    downsampled = np.zeros((new_N, new_M))

    for i in range(new_N):
        for j in range(new_M):
            block = image[i*Cx:(i+1)*Cx, j*Cy:(j+1)*Cy]

            if method == 'mean':
                downsampled[i, j] = np.mean(block)
            elif method == 'closest_to_mean':
                mean_val = np.mean(block)
                downsampled[i, j] = block.flat[np.abs(block - mean_val).argmin()]
    
    return downsampled

image = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

Cx, Cy = 2, 2

print("Remove method:")
print(downsample_image(image, Cx, Cy, method='remove'))

print("\nMean method:")
print(downsample_image(image, Cx, Cy, method='mean'))

print("\nClosest to mean method:")
print(downsample_image(image, Cx, Cy, method='closest_to_mean'))


Remove method:
[[ 1  3]
 [ 9 11]]

Mean method:
[[ 3.5  5.5]
 [11.5 13.5]]

Closest to mean method:
[[ 2.  4.]
 [10. 12.]]


# 7. Апсэмплинг

In [6]:
def upsample_image(image, Cx, Cy):
    N, M = image.shape
    new_N, new_M = N * Cx, M * Cy
    
    upsampled = np.zeros((new_N, new_M), dtype=image.dtype)

    for i in range(N):
        for j in range(M):
            upsampled[i*Cx:(i+1)*Cx, j*Cy:(j+1)*Cy] = image[i, j]

    return upsampled

image = np.array([
    [1, 2],
    [3, 4]
])

Cx, Cy = 2, 3

print("Original image:")
print(image)

print("\nUpsampled image:")
print(upsample_image(image, Cx, Cy))


Original image:
[[1 2]
 [3 4]]

Upsampled image:
[[1 1 1 2 2 2]
 [1 1 1 2 2 2]
 [3 3 3 4 4 4]
 [3 3 3 4 4 4]]


# 8. Прямое и обратное дискретное косинусное преобразование

In [88]:
def GetH():

    h = np.zeros((8, 8))
    for i in range(8):
        for j in range(8):
            if i == 0:
                h[i][j] = np.sqrt(1 / 8)
            else:
                h[i][j] = np.sqrt(2 / 8)*np.cos(np.pi * (2 * j + 1) * i / (2 * 8))

    return h

def dct(a, h):

    return np.matmul(np.matmul(h, a), np.transpose(h))

def idct(c, h):

    return np.int32(np.matmul(np.matmul(np.transpose(h), c), h))

block = np.array([
    [52, 55, 61, 66, 70, 61, 64, 73],
    [63, 59, 55, 90, 109, 85, 69, 72],
    [62, 59, 68, 113, 144, 104, 66, 73],
    [63, 58, 71, 122, 154, 106, 70, 69],
    [67, 61, 68, 104, 126, 88, 68, 70],
    [79, 65, 60, 70, 77, 68, 58, 75],
    [85, 71, 64, 59, 55, 61, 65, 83],
    [87, 79, 69, 68, 65, 76, 78, 94]
], dtype=np.int8)

h = GetH()

print("Original block:")
print(block)

dct_coeffs = dct(block, h)
print("\nDCT coefficients:")
print(dct_coeffs)

reconstructed_block = idct(dct_coeffs, h)
print("\nReconstructed block:")
print(reconstructed_block)

Original block:
[[  52   55   61   66   70   61   64   73]
 [  63   59   55   90  109   85   69   72]
 [  62   59   68  113 -112  104   66   73]
 [  63   58   71  122 -102  106   70   69]
 [  67   61   68  104  126   88   68   70]
 [  79   65   60   70   77   68   58   75]
 [  85   71   64   59   55   61   65   83]
 [  87   79   69   68   65   76   78   94]]

DCT coefficients:
[[ 5.44625000e+02 -1.25281570e+01  2.24229678e+01 -2.30451548e+01
  -7.87500000e+00  5.51608648e+01  3.22489033e+01 -8.83087347e+01]
 [-2.95054951e+01 -1.24848563e+01 -1.63727631e+01 -1.64372214e+01
  -2.08259087e+01  3.28582742e+01  9.84954618e+00 -4.22422682e+01]
 [ 1.22938053e+01 -8.94286118e+00 -1.25446417e-01  2.18948172e+01
   3.02166017e+01 -5.95939928e+01 -2.65831845e+01  7.63643435e+01]
 [ 2.09925471e+01 -7.11423694e+00 -5.67423073e+01  3.98680059e+01
   5.92869069e+01 -7.54597396e+01 -3.57968542e+01  9.83833684e+01]
 [ 1.21250000e+01 -6.55344993e+00 -1.31961210e+01 -3.95142773e+00
  -1.87500000e+00  1.7

C:\Users\Александр\AppData\Local\Temp\ipykernel_70380\2847024266.py:21: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 144 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  block = np.array([
C:\Users\Александр\AppData\Local\Temp\ipykernel_70380\2847024266.py:21: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 154 to int8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  block = np.array([


# 9. Получение матрицы квантования для заданного Q

In [18]:
import numpy as np

def get_quantization_matrix(Q):
    Q50 = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ])

    if Q < 1:
        Q = 1
    elif Q > 100:
        Q = 100

    if Q < 50:
        scale = 5000 / Q
    else:
        scale = 200 - 2 * Q

    quant_matrix = np.floor((Q50 * scale + 50) / 100).astype(int)
    quant_matrix[quant_matrix < 1] = 1
    quant_matrix[quant_matrix > 255] = 255

    return quant_matrix

quality = 90
quant_matrix = get_quantization_matrix(quality)
print("Quantization matrix for quality {}:".format(quality))
print(quant_matrix)

Quantization matrix for quality 90:
[[ 3  2  2  3  5  8 10 12]
 [ 2  2  3  4  5 12 12 11]
 [ 3  3  3  5  8 11 14 11]
 [ 3  3  4  6 10 17 16 12]
 [ 4  4  7 11 14 22 21 15]
 [ 5  7 11 13 16 21 23 18]
 [10 13 16 17 21 24 24 20]
 [14 18 19 20 22 20 21 20]]


# 10. Квантование и деквантование

In [21]:
def quantize(dct_coeffs, quant_matrix):
    return np.round(dct_coeffs / quant_matrix).astype(int)

def dequantize(quantized_coeffs, quant_matrix):
    return (quantized_coeffs * quant_matrix).astype(int)

dct_coeffs = np.array([
    [1104, -6, -24, 15, -14, 0, 0, 0],
    [-108, -8, 4, 0, 8, 0, 0, 0],
    [8, 16, 20, 7, 1, 0, 0, 0],
    [24, 10, 0, 9, 0, 1, 0, 0],
    [10, 35, 0, 0, 0, 0, 0, 0],
    [7, -11, 17, 0, 24, 0, 0, 1],
    [15, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 0]
], dtype=int)

quant_matrix = get_quantization_matrix(50)

print("Original DCT coefficients:")
print(dct_coeffs)

quantized_coeffs = quantize(dct_coeffs, quant_matrix)
print("\nQuantized coefficients:")
print(quantized_coeffs)

dequantized_coeffs = dequantize(quantized_coeffs, quant_matrix)
print("\nDequantized coefficients:")
print(dequantized_coeffs)

Original DCT coefficients:
[[1104   -6  -24   15  -14    0    0    0]
 [-108   -8    4    0    8    0    0    0]
 [   8   16   20    7    1    0    0    0]
 [  24   10    0    9    0    1    0    0]
 [  10   35    0    0    0    0    0    0]
 [   7  -11   17    0   24    0    0    1]
 [  15    0    0    0    0    0    0    0]
 [   0    0    0    1    0    1    0    0]]

Quantized coefficients:
[[69 -1 -2  1 -1  0  0  0]
 [-9 -1  0  0  0  0  0  0]
 [ 1  1  1  0  0  0  0  0]
 [ 2  1  0  0  0  0  0  0]
 [ 1  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]

Dequantized coefficients:
[[1104  -11  -20   16  -24    0    0    0]
 [-108  -12    0    0    0    0    0    0]
 [  14   13   16    0    0    0    0    0]
 [  28   17    0    0    0    0    0    0]
 [  18   44    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]]


# 11. Прямое и обратное кодирование RLE

In [96]:
def rle_encode(sequence):
    
    encoded = []
    last_val = sequence[0]
    count = 1
    for val in sequence[1:]:
        if val == last_val:
            count += 1
        else:
            encoded.append((last_val, count))
            last_val = val
            count = 1
    encoded.append((last_val, count))
    return encoded

def rle_decode(encoded_sequence):
    
    decoded = []
    for value, count in encoded_sequence:
        decoded.extend([value] * count)
    return np.array(decoded, dtype=int)

dct_coeffs = np.array([
    [1104, -6, -24, 15, -14, 0, 0, 0],
    [-108, -8, 4, 0, 8, 0, 0, 0],
    [8, 16, 20, 7, 1, 0, 0, 0],
    [24, 10, 0, 9, 0, 1, 0, 0],
    [10, 35, 0, 0, 0, 0, 0, 0],
    [7, -11, 17, 0, 24, 0, 0, 1],
    [15, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 0]
], dtype=int)

quant_matrix = get_quantization_matrix(50)
quantized_coeffs = quantize(dct_coeffs, quant_matrix)

print("Quantized coefficients:")
print(quantized_coeffs)

zigzag_sequence = zigzag_order(quantized_coeffs)
print("\nZigzag sequence:")
print(zigzag_sequence)

rle_encoded = rle_encode(zigzag_sequence)
print("\nRLE encoded sequence:")
print(rle_encoded)

rle_decoded = rle_decode(rle_encoded)
print("\nRLE decoded sequence:")
print(rle_decoded)

reconstructed_matrix = np.zeros((8, 8), dtype=int)
index = 0
bound = 8 + 8 - 1
for i in range(bound):
    if i % 2 == 0:
        r = i if i < 8 else 7
        c = 0 if i < 8 else i - 7
        while r >= 0 and c < 8:
            reconstructed_matrix[r, c] = rle_decoded[index]
            index += 1
            r -= 1
            c += 1
    else:
        r = 0 if i < 8 else i - 7
        c = i if i < 8 else 7
        while r < 8 and c >= 0:
            reconstructed_matrix[r, c] = rle_decoded[index]
            index += 1
            r += 1
            c -= 1

print("\nReconstructed matrix:")
print(reconstructed_matrix)

Quantized coefficients:
[[69 -1 -2  1 -1  0  0  0]
 [-9 -1  0  0  0  0  0  0]
 [ 1  1  1  0  0  0  0  0]
 [ 2  1  0  0  0  0  0  0]
 [ 1  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]

Zigzag sequence:
[69 -1 -9  1 -1 -2  1  0  1  2  1  1  1  0 -1  0  0  0  0  2  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]

RLE encoded sequence:
[(69, 1), (-1, 1), (-9, 1), (1, 1), (-1, 1), (-2, 1), (1, 1), (0, 1), (1, 1), (2, 1), (1, 3), (0, 1), (-1, 1), (0, 4), (2, 1), (0, 44)]

RLE decoded sequence:
[69 -1 -9  1 -1 -2  1  0  1  2  1  1  1  0 -1  0  0  0  0  2  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]

Reconstructed matrix:
[[69 -1 -2  1 -1  0  0  0]
 [-9 -1  0  0  0  0  0  0]
 [ 1  1  1  0  0  0  0  0]
 [ 2  1  0  0  0  0  0  0]
 [ 1  2  0  0  0  0  0  0]
 [ 0  0  

# 12. Функция ДКП->квантование->деквантование->обратное ДКП 

In [89]:
def process_image(img, Q):
    height, width = img.shape
    processed_img = np.zeros_like(img, dtype=np.float32)

    quant_matrix = get_quantization_matrix(Q)
    h = GetH()

    for i in range(0, height, 8):
        for j in range(0, width, 8):
            block = img[i:i+8, j:j+8]
            dct_block = dct(block, h)
            quantized_block = quantize(dct_block, quant_matrix)
            dequantized_block = dequantize(quantized_block, quant_matrix)
            idct_block = idct(dequantized_block, h)
            processed_img[i:i+8, j:j+8] = idct_block

    processed_img = np.clip(processed_img, 0, 255).astype(np.uint8)

    return processed_img

# 13. Функция разбиения изображения на каналы

In [63]:
def imgtorawchanels(img):
    imgarr = np.array(img)
    R = []; G = []; B = []
    for line in imgarr:
        for pixel in line:
            R.append(pixel[0])
            G.append(pixel[1])
            B.append(pixel[2])
    return np.reshape(R, (len(imgarr), len(imgarr[0]))), np.reshape(G, (len(imgarr), len(imgarr[0]))), np.reshape(B, (len(imgarr), len(imgarr[0])))

In [91]:
img = Image.open("cat.jpeg")
img = img.resize((400,400))
img_array = np.array(img)
R,G,B = imgtorawchanels(img)
for Q in [90, 70, 50, 30, 10, 1]:
    processed_R = process_image(R, Q)
    processed_G = process_image(G, Q)
    processed_B = process_image(B, Q)
    
    res_rgb = []
    rpR = np.reshape(processed_R, len(img_array) * len(img_array[0]))
    rpG = np.reshape(processed_G, len(img_array) * len(img_array[0]))
    rpB = np.reshape(processed_B, len(img_array) * len(img_array[0]))
    for j in range(len(rpR)):
        res_rgb.append([rpR[j],rpG[j],rpB[j]])
    res_rgb = np.array(res_rgb)
    res_rgb = np.reshape(res_rgb,(len(img_array),len(img_array[0]), 3))
    immg = Image.fromarray(res_rgb.astype(np.uint8))
                     
    immg.show()
    immg.save(f"./quality/cat_quality{Q}.jpeg")
    immg.close()

# 14. Запись RLE в файл

In [135]:
def rle_to_file(encoded_data, filename):
    with open(filename, 'wb') as file:
        for item in encoded_data:
            file.write(int(item[0]).to_bytes(2, byteorder='big', signed=True))
            file.write(int(item[1]).to_bytes(2, byteorder='big', signed=False))
    return

# 15. Чтение RLE из файла

In [136]:
def rle_from_file(filename):
    encoded_data = []
    with open(filename, 'rb') as file:
        while True:
            num = file.read(2)
            if not num:
                break
            count = file.read(2)
            encoded_data.append((int.from_bytes(num, "big", signed=True), int.from_bytes(count, "big", signed=False)))
    return encoded_data

In [110]:
rle_to_file(rle_encoded, "test_rle")

In [114]:
rle_decoded = rle_decode(rle_from_file("test_rle"))

In [115]:
reconstructed_matrix = np.zeros((8, 8), dtype=int)
index = 0
bound = 8 + 8 - 1
for i in range(bound):
    if i % 2 == 0:
        r = i if i < 8 else 7
        c = 0 if i < 8 else i - 7
        while r >= 0 and c < 8:
            reconstructed_matrix[r, c] = rle_decoded[index]
            index += 1
            r -= 1
            c += 1
    else:
        r = 0 if i < 8 else i - 7
        c = i if i < 8 else 7
        while r < 8 and c >= 0:
            reconstructed_matrix[r, c] = rle_decoded[index]
            index += 1
            r += 1
            c -= 1

print("\nReconstructed matrix:")
print(reconstructed_matrix)


Reconstructed matrix:
[[69 -1 -2  1 -1  0  0  0]
 [-9 -1  0  0  0  0  0  0]
 [ 1  1  1  0  0  0  0  0]
 [ 2  1  0  0  0  0  0  0]
 [ 1  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


# 16. Компрессор

In [137]:
def process_compression(matrix, Q):
    height, width = matrix.shape
    processed_matrix = np.zeros_like(matrix, dtype=np.float32)

    quant_matrix = get_quantization_matrix(Q)
    h = GetH()

    for i in range(0, height, 8):
        for j in range(0, width, 8):
            block = matrix[i:i+8, j:j+8]
            dct_block = dct(block, h)
            quantized_block = quantize(dct_block, quant_matrix)
            processed_matrix[i:i+8, j:j+8] = quantized_block

    return processed_matrix

def compressor(filename, Cx, Cy, method='remove', Q=50):
    image = Image.open(filename)
    array = imgtoraw(image)
    y,Cb,Cr = rgbtoycbcr(array)
    res_ycbcr = []
    for i in range(len(y)):
        res_ycbcr.append([y[i],Cb[i],Cr[i]])
    res_ycbcr = np.array(res_ycbcr)
    res_ycbcr = np.reshape(res_ycbcr,(len(array),len(array[0]),3))
    image_ycbcr = Image.fromarray(res_ycbcr.astype(np.uint8), mode='YCbCr')
    image_ycbcr.save(f"./{filename}_output/{filename}_ycbcr.jpeg")
    
    downsampled_y = downsample_image(np.reshape(y, (len(array),len(array[0]))), Cx, Cy, method)
    downsampled_Cb = downsample_image(np.reshape(Cb, (len(array),len(array[0]))), Cx, Cy, method)
    downsampled_Cr = downsample_image(np.reshape(Cr, (len(array),len(array[0]))), Cx, Cy, method)
    res_downsampled = []
    for i in range(len(array)//Cy*len(array[0])//Cx):
        res_downsampled.append([np.reshape(downsampled_y, (len(array)//Cy*len(array[0])//Cx))[i],np.reshape(downsampled_Cb, (len(array)//Cy*len(array[0])//Cx))[i],np.reshape(downsampled_Cr, (len(array)//Cy*len(array[0])//Cx))[i]])
    res_downsampled = np.array(res_downsampled)
    res_downsampled = np.reshape(res_downsampled,(len(array)//Cy,len(array[0])//Cx,3))
    image_downsampled = Image.fromarray(res_downsampled.astype(np.uint8), mode='YCbCr')
    image_downsampled.save(f"./{filename}_output/{filename}_downsampled.jpeg")
    
    quantized_y = process_compression(np.reshape(downsampled_y, (len(array)//Cy,len(array[0])//Cx)), Q)
    quantized_Cb = process_compression(np.reshape(downsampled_Cb, (len(array)//Cy,len(array[0])//Cx)), Q)
    quantized_Cr = process_compression(np.reshape(downsampled_Cr, (len(array)//Cy,len(array[0])//Cx)), Q)
                                      
    zigzag_y = zigzag_order(quantized_y)
    zigzag_Cb = zigzag_order(quantized_Cb)
    zigzag_Cr = zigzag_order(quantized_Cr)
                                      
    rle_y = rle_encode(zigzag_y)
    rle_Cb = rle_encode(zigzag_Cb)
    rle_Cr = rle_encode(zigzag_Cr)
                                      
    rle_to_file(rle_y, f"./{filename}_output/{filename}_y_rle")
    rle_to_file(rle_Cb, f"./{filename}_output/{filename}_Cb_rle")
    rle_to_file(rle_Cr, f"./{filename}_output/{filename}_Cr_rle")

In [138]:
compressor("cat_resized.jpeg", 2, 2)

# 17. Декомпрессор

In [153]:
def process_decompression(matrix, Q):
    height, width = matrix.shape
    processed_matrix = np.zeros_like(matrix, dtype=np.float32)

    quant_matrix = get_quantization_matrix(Q)
    h = GetH()

    for i in range(0, height, 8):
        for j in range(0, width, 8):
            block = matrix[i:i+8, j:j+8]
            dequantized_block = dequantize(block, quant_matrix)
            idct_block = idct(dequantized_block, h)
            processed_matrix[i:i+8, j:j+8] = idct_block

    return processed_matrix

def decompressor(filename, filename_y, filename_Cb, filename_Cr, height, weight, Q, Cx, Cy):
    rle_y = rle_from_file(filename_y)
    rle_Cb = rle_from_file(filename_Cb)
    rle_Cr = rle_from_file(filename_Cr)
    
    rle_y_decoded = rle_decode(rle_y)
    rle_Cb_decoded = rle_decode(rle_Cb)
    rle_Cr_decoded = rle_decode(rle_Cr)
    
    y = inverse_zigzag_order(rle_y_decoded, height, weight)
    Cb = inverse_zigzag_order(rle_Cb_decoded, height, weight)
    Cr = inverse_zigzag_order(rle_Cr_decoded, height, weight)
    
    y_idct = process_decompression(np.reshape(y, (height, weight)), Q)
    Cb_idct = process_decompression(np.reshape(Cb, (height, weight)), Q)
    Cr_idct = process_decompression(np.reshape(Cr, (height, weight)), Q)
    
    y_upsampled = upsample_image(y_idct, Cx, Cy)
    Cb_upsampled = upsample_image(Cb_idct, Cx, Cy)
    Cr_upsampled = upsample_image(Cr_idct, Cx, Cy)
    res_upsampled = []
    for i in range(height*Cy*weight*Cx):
        res_upsampled.append([np.reshape(y_upsampled, height*Cy*weight*Cx)[i],np.reshape(Cb_upsampled, height*Cy*weight*Cx)[i],np.reshape(Cr_upsampled, height*Cy*weight*Cx)[i]])
    res_upsampled = np.array(res_upsampled)
    height*=Cy
    weight*=Cx
    res_upsampled = np.reshape(res_upsampled,(height,weight,3))
    image_upsampled = Image.fromarray(res_upsampled.astype(np.uint8), mode='YCbCr')
    image_upsampled.save(f"./{filename}_output/{filename}_upsampled.jpeg")
    
    R,G,B = ycbcrtorgb(res_upsampled)
    res_rgb = []
    for j in range(len(R)):
        res_rgb.append([R[j],G[j],B[j]])
    res_rgb = np.array(res_rgb)
    res_rgb = np.reshape(res_rgb,(height,weight,3))
    image_rgb = Image.fromarray(res_rgb.astype(np.uint8), mode='RGB')
    image_rgb.save(f"./{filename}_output/{filename}_rgb.jpeg")

In [154]:
decompressor("cat_resized.jpeg", "./cat_resized.jpeg_output/cat_resized.jpeg_y_rle", "./cat_resized.jpeg_output/cat_resized.jpeg_Cb_rle", "./cat_resized.jpeg_output/cat_resized.jpeg_Cr_rle", 200, 200, 50, 2, 2)